# astroVIPER Tutorial: Image

In [1]:
import os

from importlib.metadata import version

try:
    import astroviper

    print("astroVIPER version", version("astroviper"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing astroVIPER")

    os.system("pip install astroviper")

    import xradio

    print("astroVIPER version", version("astroviper"), " installed.")

astroVIPER version 0.0.6 already installed.


## Download dataset

In [2]:
from xradio.data.datasets import download

imname = "demo_simulated.im"
download(imname)

File exists.


## Setup Dask Cluster
To simplify things we are going to start of by just using a single thread (everything will run in serial).

In [3]:
# import astroviper
# from astroviper.client import local_client

# viper_client = local_client(cores=2, memory_limit="4GB")
# viper_client

import dask

dask.config.set(scheduler="synchronous")

## Inspect Image Dataset

In [4]:
from xradio.image import load_image, read_image, write_image

img_xds = read_image(infile="demo_simulated.im", chunks={"l": 40, "m": 20, "freq": 5})
img_xds

Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


<xarray.Dataset>
Dimensions:          (time: 1, polarization: 4, frequency: 50, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 5.154e+04
  * polarization     (polarization) <U1 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 1.415e+09 1.415e+09 ... 1.415e+09
    vel              (frequency) float64 1.146e+06 1.146e+06 ... 1.136e+06
    right_ascension  (l, m) float64 0.0291 0.0291 0.0291 ... 6.254 6.254 6.254
    declination      (l, m) float64 -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 'major' 'minor' 'pa'
Dimensions without coordinates: l, m
Data variables:
    sky              (time, polarization, frequency, l, m) float64 dask.array<chunksize=(1, 4, 50, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool dask.array<chunksize=(1, 4, 50, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 dask.array<chunksize=(1, 4, 50, 3), meta=np.ndarray>
Attributes:
    direction:        {'system': 'FK5', 'equinox': 'J2000', 'conversion_syste...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'refe...
    obsdate:          {'time_scale': 'UTC', 'unit': 'd', 'value': 51544.00000...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      
    user:             {}
    active_mask:      mask0
    description:      None
    history:          <xarray.Dataset>\nDimensions:    (row: 15, TIME: 15, PR...

In [5]:
## Create Parallel Coordinates, Map

In [6]:
import numpy as np
import xarray as xr
import dask
from astroviper._concurrency._graph_tools import (
    _map,
    _reduce,
    _make_time_coord,
    _make_frequency_coord,
    _make_parallel_coord,
)
from astroviper._domain._image._moment_max import _moment_max
from astroviper._domain._imaging import _make_image
from xradio.image import load_image, read_image, write_image
from astroviper._utils._display import _dict_to_html
from IPython.display import display, HTML

input_parms = {}
parallel_coords = {}
parallel_coords["frequency"] = _make_parallel_coord(coord=img_xds.frequency, n_chunks=6)
sel_parms = {}

input_data_name = "demo_simulated.im"


def test_func(input_parms):
    img_xds = load_image(
        input_parms["input_data_name"], block_des=input_parms["data_sel"]["image"]
    )
    display(HTML(_dict_to_html(input_parms)))
    print("****")


graph = _map(
    input_data_name,
    input_data_type="image",
    parallel_coords=parallel_coords,
    func_chunk=test_func,
    input_parms=input_parms,
    client=None,
)
dask.compute(graph)

Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****
Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


****


(([None, None, None, None, None, None], None),)